# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 1 ou 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [1]:
%%capture

#Instalando o tweepy
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [2]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle


___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: ***@Giiovana_lemes***


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret
    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [3]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @Giiovana_lemes

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [4]:
#Produto escolhido:
produto = 'Trakinas'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [5]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [6]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

In [7]:
#se quiser abrir a pasta que está sendo usada
#! explorer .


___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [8]:
%matplotlib inline
from mpmath import mpf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
plt.style.use('ggplot')
import mpmath

dados_treinamento = pd.read_excel('Trakinas.xlsx', "Treinamento", sep=',')

In [9]:
#para pegar os dados da aba Treinamento
dados_treinamento = pd.read_excel('Trakinas.xlsx', sheetname="Treinamento")
dados_treinamento = dados_treinamento.loc[:,["Treinamento", "Classificação"]]

In [10]:
#para fazer value counts ta aba Treinamento
vc_treinamento = dados_treinamento.Classificação.value_counts(True)

In [11]:
#Probabilidade do tweet ser relevante
prob_tweet_relevante = vc_treinamento.loc["relevante"]
prob_tweet_relevante

0.083333333333333329

In [12]:
#Probabilidade do tweet ser irrelevante
prob_tweet_irrelevante = vc_treinamento.loc["irrelevante"]
prob_tweet_irrelevante

0.91666666666666663

In [13]:
#para deixar todos os tweets como se fossem um único tweet "gigante"
tweet_gigante_treinamento = np.sum(dados_treinamento.Treinamento + " ") 

tweet_gigante_treinamento_irrel = np.sum(dados_treinamento[dados_treinamento.Classificação=="irrelevante"].Treinamento + " ")
tweet_gigante_treinamento_rel = np.sum(dados_treinamento[dados_treinamento.Classificação=="relevante"].Treinamento + " ")

#tweet_gigante_teste = np.sum(dados_teste.Teste + " ") 

qtd_relevante_treinamento = len(dados_treinamento[dados_treinamento.Classificação=="relevante"])
qtd_irrelevante_treinamento = len(dados_treinamento[dados_treinamento.Classificação=="irrelevante"])
qtd_total_tweets = qtd_relevante_treinamento + qtd_irrelevante_treinamento


In [43]:
#TIRAR A PONTUAÇÃO
import string

exclude = set(string.punctuation)  #pontuation pega todas as pontuaçoes 
def SemPontuacao(tweet_gigante):
    tweet_gigante = tweet_gigante.split() #para cada palavra do tweet gigante virar um tweet de palavra única
    lista_cada_palavra = []
    for s in tweet_gigante:
        for i in s:
            if i in exclude:
                s = s.replace(i,'') 
        lista_cada_palavra.append(s) #volta a ser um tweet gigante
    return lista_cada_palavra   
    
gigante_irrelevante = SemPontuacao(tweet_gigante_treinamento_irrel)
gigante_relevante = SemPontuacao(tweet_gigante_treinamento_rel)

gigante = SemPontuacao(tweet_gigante_treinamento)




In [44]:
gigante

['nível',
 'de',
 'anormalidade',
 'metade',
 'da',
 'barba',
 'loira',
 'e',
 'metade',
 'morena',
 'sou',
 'quase',
 'uma',
 'trakinas',
 'meio',
 'a',
 'meio',
 'rt',
 'ironicovcs',
 'msg',
 'boa',
 'é',
 'aquela',
 'que',
 'te',
 'faz',
 'ficar',
 'lendo',
 'dnv',
 'de',
 '5',
 'em',
 '5',
 'minutos',
 'rt',
 'repousando',
 'a',
 'vida',
 'é',
 'um',
 'baile',
 'ou',
 'tu',
 'segue',
 'ou',
 'tu',
 'dança',
 'rt',
 'giovanardileo',
 'é',
 'muito',
 'vacilo',
 'falsidade',
 'mimimi',
 'cobrança',
 'e',
 'má',
 'fé',
 '',
 'por',
 'isso',
 'cada',
 'vez',
 'mais',
 'considero',
 'menos',
 'pessoas',
 '🎭',
 'rt',
 'hemillysantos1',
 'e',
 'ainda',
 'tem',
 'pessoas',
 'que',
 'me',
 'chamam',
 'de',
 'trakinas',
 '😪',
 'httpstco7xoinxr4xf',
 'osergiosaiu',
 'e',
 'tudo',
 'começou',
 'com',
 'trakinas',
 'rt',
 'iloveputarias',
 'como',
 'desmascarar',
 'um',
 'maconheiro',
 '',
 'vc',
 'fuma',
 '',
 'o',
 'que',
 'tô',
 'amassando',
 'um',
 'trakinas',
 'meio',
 'a',
 'meio',
 'docin

In [51]:
#para o tweet gigante virar uma Series (tabela)
palavras_irrelevante = pd.Series(gigante_irrelevante)
palavras_relevante = pd.Series(gigante_relevante)

#todas as palavras que aparecem relevantes e irrelevantes
palavras_totais = pd.Series(gigante)


In [17]:
prob_total = pd.DataFrame(dados_treinamento.Classificação.value_counts(True)) #probabilidade total irrelevante e relevante
prob_tweet_irrelevante = prob_total.Classificação[0] #prob total irrelevantes
prob_tweet_relevante = prob_total.Classificação[1] #prob total relevante

prob_total

,Classificação
irrelevante,0.916667
relevante,0.083333


In [58]:
qtd_irrelevante = len(dados_treinamento[dados_treinamento.Classificação=="irrelevante"]) #Quantidade de irrelevantes
qtd_relevante = len(dados_treinamento[dados_treinamento.Classificação=="relevante"]) #Quantidade de relevantes
qtd_irrelevante

275

In [19]:
#para saber quantas palavras diferentes existem entre relevantes e irrelevantes no TREINAMENTO
palavras_diferentes = len(palavras_totais.value_counts())

palavras_diferentes_relevante = len(palavras_relevante.value_counts())
palavras_diferentes_irrelevante = len(palavras_irrelevante.value_counts())

PP_relevante  = palavras_relevante.value_counts()
PP_irrelevante = palavras_irrelevante.value_counts()
palavras_dif = len(palavras_totais.value_counts())


#cada palavra e quantas vezes essa paravra apareceu
palavras_diferentes_relevante_qtd = (palavras_relevante.value_counts())
palavras_diferentes_irrelevante_qtd = (palavras_irrelevante.value_counts())

In [20]:
palavras_diferentes_irrelevante_qtd["trakinas"]

109

In [21]:
#para mostrar a palavra e quantas vezes ela aparece
for palavra in palavras_diferentes_relevante_qtd.index:
    print("palavra ", palavra, " aparece ", palavras_diferentes_relevante_qtd[palavra], " vezes")

palavra  trakinas  aparece  23  vezes
palavra  de  aparece  12  vezes
palavra  mini  aparece  6  vezes
palavra  um  aparece  6  vezes
palavra  era  aparece  5  vezes
palavra  meio  aparece  4  vezes
palavra  a  aparece  4  vezes
palavra  bom  aparece  3  vezes
palavra  limão  aparece  3  vezes
palavra  essa  aparece  3  vezes
palavra  queria  aparece  3  vezes
palavra  ❤️  aparece  3  vezes
palavra  melhor  aparece  3  vezes
palavra  volta  aparece  3  vezes
palavra  muito  aparece  3  vezes
palavra  morango  aparece  2  vezes
palavra  gente  aparece  2  vezes
palavra  chocolate  aparece  2  vezes
palavra  amo  aparece  2  vezes
palavra  é  aparece  2  vezes
palavra  do  aparece  2  vezes
palavra  ❤  aparece  2  vezes
palavra  só  aparece  2  vezes
palavra  o  aparece  2  vezes
palavra  e  aparece  2  vezes
palavra  eu  aparece  2  vezes
palavra  esse  aparece  2  vezes
palavra  te  aparece  2  vezes
palavra  sdds  aparece  2  vezes
palavra  rt  aparece  2  vezes
palavra  você  aparece

## Criando classificador para o treinamento 

### Calculando a probabilidade de cada palavra aparecer (em relevante em irrelevante)

In [60]:
#para calcular a propabilidade de cada palavra aparecer em Relevante
P_palavra_relevante = (PP_relevante+1)/(qtd_relevante + palavras_dif) #lista com probabilidade das palavras quando tw relevante

#para calcular a propabilidade de cada palavra aparecer em Irrelevante
P_palavra_irrelevante = (PP_irrelevante+1)/(qtd_irrelevante + palavras_dif) #lista com probabilidade das palavras quando tw neutro

P_palavra_relevante

trakinas     0.018898
de           0.010236
mini         0.005512
um           0.005512
era          0.004724
meio         0.003937
a            0.003937
bom          0.003150
limão        0.003150
essa         0.003150
queria       0.003150
❤️           0.003150
melhor       0.003150
volta        0.003150
muito        0.003150
morango      0.002362
gente        0.002362
chocolate    0.002362
amo          0.002362
é            0.002362
do           0.002362
❤            0.002362
só           0.002362
o            0.002362
e            0.002362
eu           0.002362
esse         0.002362
te           0.002362
sdds         0.002362
rt           0.002362
               ...   
nunca        0.001575
ver          0.001575
terra        0.001575
pote         0.001575
esqueça      0.001575
😭            0.001575
das          0.001575
esqueceu     0.001575
deus         0.001575
parabéns     0.001575
gostosin     0.001575
😋            0.001575
universo     0.001575
tô           0.001575
amassando 

In [23]:
####################################

In [24]:
# #para calcular a propabilidade de cada palavra aparecer em Relevante
# lista_palavras_rel = []
# lista_prob_rel = []
# for palavra in palavras_diferentes_relevante_qtd.index:
    
#     prob_palavra_relevante = ( palavras_diferentes_relevante_qtd[palavra] + 1 ) / ( len(palavras_relevante) + palavras_diferentes_relevante)
#     lista_palavras_rel.append(palavra)
#     lista_prob_rel.append(prob_palavra_relevante)
#     #print("palavra ", palavra, " aparece ", palavras_diferentes_relevante_qtd[palavra], " vezes")
#     #print ("a probabilidade dela aparecer em RELEVANTE é:", prob_palavra_relevante )
#     #print ("__________")

    

    

In [25]:

# #Criando um dicionário com as palavras e suas probabilidades DEF   
# dicionario_relevantes = dict()
# dicionario_irrelevantes = dict()
# def Dic(lista_palavra,lista_prob,dic):
#     for i in range(len(lista_palavra)):
#         dic[lista_palavra[i]]= lista_prob[i]
#     return(dic)


In [26]:
# #Usando a def para criar um dicionário de palavras e suas prob relevantes
# prob_p_relevante = Dic(lista_palavras_rel,lista_prob_rel,dicionario_relevantes)
# prob_p_relevante

In [27]:
##########################################

### Irrelevante (probabilidade de cada palavra aparecer)

In [28]:
# #para calcular a propabilidade de cada palavra aparecer em Irrelevante
# lista_palavras_irrel = []
# lista_prob_irrel = []
# for palavra in palavras_diferentes_irrelevante_qtd.index:
    
#     prob_palavra_irrelevante = ( palavras_diferentes_irrelevante_qtd[palavra] + 1 ) / ( len(palavras_irrelevante) + palavras_diferentes_irrelevante)
#     lista_palavras_irrel.append(palavra)
#     lista_prob_irrel.append(prob_palavra_irrelevante)

In [29]:
# prob_p_irrelevante = Dic(lista_palavras_irrel,lista_prob_irrel,dicionario_irrelevantes)
# prob_p_irrelevante

# Analisando Teste

In [30]:
#para pegar os dados da aba Teste
dados_teste = pd.read_excel('Trakinas.xlsx', "Teste", sep=',')

dados_teste = pd.read_excel('Trakinas.xlsx', sheetname="Teste")
dados_teste = dados_teste.loc[:,["Teste", "Classificação"]]

In [31]:
#para fazer value counts ta aba teste
vc_teste = dados_teste.Classificação.value_counts(True)

In [32]:
#Probabilidade do tweet ser relevante
vc_teste.loc["relevante"]

0.040000000000000001

In [33]:
#Probabilidade do tweet ser irrelevante
vc_teste.loc["irrelevante"]

0.95999999999999996

In [34]:
lista_teste = dados_teste.Teste

In [70]:
#Tirando e pontuação e separando cada tweet em uma lista e depois, cada palavra desse tweet dentro dessa lista

exclude = set(string.punctuation)  #pontuation pega todas as pontuaçoes 

def SemPontuacao(tweet_gigante):
    lista_cada_palavra_do_tweet = []
    for g in range(len(tweet_gigante)):
        lista_tweets_splitados = tweet_gigante[g].split()
        lista_tweets = []

        for f in lista_tweets_splitados:
            for i in f:
                if i in exclude:
                    f = f.replace(i,'') 
                
            lista_tweets.append(f)
        lista_cada_palavra_do_tweet.append(lista_tweets)
                
    return lista_cada_palavra_do_tweet   

lista_palavras_tweet = SemPontuacao (lista_teste)


#cria lista sem pontuação
teste_limpo = SemPontuacao(lista_teste)

#####

def Prob_cada_tweet(listafrase,P_P,P_,T_):
    listaP = []
    for frase in listafrase:
        P_frase = mpmath.mpf(1.000)
        for p in frase:
            if p in P_P:
                P_frase = P_frase * P_P[p] * P_
            else:
                P_frase = mpmath.mpf(1)/(T_+palavras_diferentes) * P_frase * P_
        listaP.append(P_frase)
    return listaP


##Cálculo da probabilidade do tweet ser irrelevante
teste_prob_irrelevante = Prob_cada_tweet(teste_limpo,P_palavra_irrelevante,prob_tweet_irrelevante,qtd_irrelevante)

#Cálculo da probabilidade do tweet ser relevante
teste_prob_relevante = Prob_cada_tweet(teste_limpo,P_palavra_relevante,prob_tweet_relevante,qtd_relevante)


In [85]:
#agora temos que comparar a lista de probabilidade feita no teste, e ver qual prevalece, se é neutro ou relevante
def Comparador(teste_prob_irrelevante,teste_prob_relevante):
    resultado = []
    for i in range(len(teste_prob_irrelevante)):
        if teste_prob_irrelevante[i] > teste_prob_relevante[i]:
            resultado.append('irrelevante')
        elif teste_prob_irrelevante[i] < teste_prob_relevante[i]:
            resultado.append('relevante')
    return resultado

#lista de respostas que esse codigo vai gerar
resultado_teste = Comparador(teste_prob_irrelevante, teste_prob_relevante)

len(resultado_teste)

200

In [82]:
resultado_teste

['neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',
 'neutro',

In [86]:
data = {'Tweets': teste_limpo, 'Classificação': dados_teste.Classificação, 'Respostas': resultado_teste}


In [87]:
tabelaT = pd.DataFrame(data, columns = ['Tweets','Classificação','Respostas'])
tabelaT

,Tweets,Classificação,Respostas
0,"[te, jurupinga, e, amor, também, pq, nois, é, ...",irrelevante,irrelevante
1,"[rt, aqueleeita, realmente, acho, triste, essa...",irrelevante,irrelevante
2,"[ontem, tia, luna, veio, aqui, e, foi, tão, en...",irrelevante,irrelevante
3,"[na, vdd, eu, sei, um, pouco]",irrelevante,irrelevante
4,"[amo, me, que, marquem, nas, publicações, 💙]",irrelevante,irrelevante
5,"[fico, vendo, as, fotos, desses, casais, e, fi...",irrelevante,irrelevante
6,"[sim, jshshs, httpstcotvnnob1ufz]",irrelevante,irrelevante
7,"[rt, planetasapa, a, gente, morre, de, saudade...",irrelevante,irrelevante
8,"[wendeleek, ela, é, linda]",irrelevante,irrelevante
9,"[rt, umfilosofocitou, eu, confio, desconfiando...",irrelevante,irrelevante


___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [88]:
verificacao = []
for i in range(len(resultado_teste)):
    if dados_teste.Classificação[i] == resultado_teste[i]:
        if resultado_teste[i] == 'irrelevante':
            verificacao.append('negativo verdadeiro')
        elif resultado_teste[i] == 'relevante':
            verificacao.append('positivo verdadeiro')
            
    elif dados_teste.Classificação[i] != resultado_teste[i]:
        if resultado_teste[i] == 'irrelevante':
            verificacao.append('negativo falso')
        elif resultado_teste[i] == 'relevante':
            verificacao.append('positivo falso')
    

In [89]:
verificacao_final = pd.Series(verificacao)
porcentagem = verificacao_final.value_counts(True)
porcentagem

negativo verdadeiro    0.96
negativo falso         0.04
dtype: float64

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
